In [1]:
import gensim
import pandas as pd
import logging
import string

from spacy.lang import sk
from spacy.tokenizer import Tokenizer
from spacy.lang.sk import Slovak

nlp = Slovak()
tokenizer = Tokenizer(nlp.vocab)


stop_words = sk.STOP_WORDS
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
df =pd.read_csv("csvs/tokens.csv")

/home/jurkis/leto2021/colbert/objax/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
class MyCorpus:
    __slots__ = ['stop_words',"nlp","tokenizer"]
    
    def __init__(self):
        self.stop_words = sk.STOP_WORDS
        self.nlp = nlp
        self.tokenizer = tokenizer 
    def __iter__(self):
        for d in df['toks']:
                    doc = [ x.text for x in self.tokenizer(d.lower())
                            if x.text not in self.stop_words ]
                    yield doc
    


In [3]:
c = MyCorpus()
dictionary = gensim.corpora.Dictionary(c)

2021-04-19 12:04:00,794 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-04-19 12:06:08,857 : INFO : built Dictionary(2374769 unique tokens: ['\n', '(ako', '(ač', '(nepomucký),', '(organu)']...) from 2147 documents (total 26827723 corpus positions)
2021-04-19 12:06:08,868 : DEBUG : starting a new internal lifecycle event log for Dictionary
2021-04-19 12:06:08,868 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(2374769 unique tokens: ['\\n', '(ako', '(ač', '(nepomucký),', '(organu)']...) from 2147 documents (total 26827723 corpus positions)", 'datetime': '2021-04-19T12:06:08.857846', 'gensim': '4.0.1', 'python': '3.8.5 (default, Jan 27 2021, 15:41:15) \n[GCC 9.3.0]', 'platform': 'Linux-5.4.0-70-generic-x86_64-with-glibc2.29', 'event': 'created'}


In [5]:
stop_ids = [dictionary.token2id[stopword] 
            for stopword in c.stop_words 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid,docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(once_ids)

dictionary.compactify()

2021-04-19 12:07:24,474 : DEBUG : rebuilding dictionary, shrinking gaps
2021-04-19 12:07:25,120 : DEBUG : rebuilding dictionary, shrinking gaps


In [6]:
from gensim import models
tfidf = models.TfidfModel(( dictionary.doc2bow(doc) for doc in c  ))
bow_corpus = ( dictionary.doc2bow(doc) for doc in c )
corpus_tfidf = tfidf[bow_corpus]
lsi_model = models.LsiModel(corpus_tfidf,id2word=dictionary,num_topics=150 )
corpus_lsi = lsi_model[corpus_tfidf]
lsi_model.print_topics(100)

2021-04-19 12:08:28,530 : INFO : collecting document frequencies
2021-04-19 12:08:28,537 : INFO : PROGRESS: processing document #0
2021-04-19 12:09:55,159 : DEBUG : starting a new internal lifecycle event log for TfidfModel
2021-04-19 12:09:55,160 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 2147 documents and 841214 features (10290363 matrix non-zeros)', 'datetime': '2021-04-19T12:09:55.159177', 'gensim': '4.0.1', 'python': '3.8.5 (default, Jan 27 2021, 15:41:15) \n[GCC 9.3.0]', 'platform': 'Linux-5.4.0-70-generic-x86_64-with-glibc2.29', 'event': 'initialize'}
2021-04-19 12:09:55,213 : INFO : using serial LSI version on this node
2021-04-19 12:09:55,214 : INFO : updating model with new documents
2021-04-19 12:12:15,481 : INFO : preparing a new chunk of documents
2021-04-19 12:12:15,482 : DEBUG : converting corpus to csc format
2021-04-19 12:12:39,405 : INFO : using 100 extra samples and 2 power iterations
2021-04-19 12:12:39,406 : INFO : 1st phase: construct

[(0,
  '0.998*"\xa0" + 0.032*"\n\xa0\xa0" + 0.016*"\n\n" + 0.009*"se" + 0.009*"\n\xa0\xa0\xa0\xa0" + 0.008*"\xa0\n" + 0.007*"\n\xa0\xa0\xa0" + 0.006*"\n\xa0\xa0\xa0\xa0\xa0" + 0.005*"." + 0.005*"\n\n\n\n \n"'),
 (1,
  '0.726*"se" + 0.170*"jsem" + 0.142*"když" + 0.125*"byl" + 0.121*"jako" + 0.115*"co" + 0.108*"jsou" + 0.106*"ve" + 0.101*"jen" + 0.093*"ještě"'),
 (2,
  '0.270*"se" + -0.120*"\n\n" + -0.103*"—" + -0.074*"„a" + -0.074*"veľmi" + -0.066*"pani" + 0.064*"jsem" + -0.062*"pán" + -0.059*"\n\n \n" + -0.057*"akoby"'),
 (3,
  '-0.613*"adalb." + -0.437*"\n\n" + -0.386*"č." + -0.156*"\n\n \n" + -0.143*"\n\n \n\n" + -0.123*"\n\n\n\n" + -0.111*"(pozri" + -0.110*"\n \n\n" + -0.062*"bošácka" + -0.060*"jak"'),
 (4,
  '0.233*"r." + -0.146*"\n\n" + 0.122*"roku" + 0.095*"slovenského" + -0.091*"adalb." + 0.085*"sv." + 0.084*"slovenskej" + -0.082*"„a" + 0.081*"národa" + 0.077*"slovenských"'),
 (5,
  '0.531*"\n\n" + -0.433*"adalb." + -0.303*"č." + 0.296*"\n\n\n\n" + 0.138*"jak" + 0.130*"\n " + 0.

In [9]:
# from toolz.itertoolz import concat

sum(map(len,df['toks']))

213753842